# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32), 32768 total points
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.123828e+02     1.515170e+00
 * time: 0.43060803413391113
     1     1.096319e+01     9.424041e-01
 * time: 1.0360639095306396
     2    -1.212317e+01     9.387949e-01
 * time: 1.106600046157837
     3    -3.424970e+01     7.644800e-01
 * time: 1.2273139953613281
     4    -4.788737e+01     5.520322e-01
 * time: 1.311527967453003
     5    -5.703239e+01     2.188563e-01
 * time: 1.3976070880889893
     6    -5.988656e+01     1.204298e-01
 * time: 1.4689109325408936
     7    -6.087511e+01     5.685124e-02
 * time: 1.5521430969238281
     8    -6.125037e+01     9.120279e-02
 * time: 1.6146910190582275
     9    -6.159172e+01     3.396612e-02
 * time: 1.6777489185333252
    10    -6.182435e+01     2.702401e-02
 * time: 1.7410149574279785
    11    -6.199526e+01     2.161100e-02
 * time: 1.8048570156097412
    12    -6.206808e+01     1.758027e-02
 * time: 1.8682420253753662
    13    -6.213269e+01     1.335476e-02
 * time: 1.94233298301

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671064
    AtomicLocal         -18.8557693
    AtomicNonlocal      14.8522663
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485372 
    Xc                  -19.3336820

    total               -62.261666460462